In [11]:
import gurobipy as gp
from gurobipy import GRB

def _get_b_node(b_node, i, k):
    """
    Helper to read b_{ik}. Supports:
      - dict[(i,k)] -> value
      - list-of-lists b_node[i][k]
    """
    if isinstance(b_node, dict):
        return float(b_node.get((i, k), 0.0))
    return float(b_node[i][k])

def _get_b_supply(b_supply, k):
    """
    Helper to read b_k. Supports:
      - dict[k] -> value
      - list/tuple b_supply[k]
    """
    if isinstance(b_supply, dict):
        return float(b_supply[k])
    return float(b_supply[k])

def create_master_problem(
    V,
    E,
    P_list,
    R,
    b_node,
    b_supply,
    M_prec=1e6,
    M_flow=1e6,
    gurobi_output=False,
):
    """
    Build the initial master problem for a given scenario set P_list.

    V       : list of nodes (0 is source, V[-1] is sink n+1)
    E       : list of fixed precedence arcs (i,j) -> x_ij = 1
    P_list  : list of duration vectors p^h; each p^h has len(p^h) == len(V)
    R       : list of resource indices (e.g. [0,1,...])
    b_node  : node-by-resource balances b_{ik}, as dict[(i,k)] or 2D list [i][k]
    b_supply: per-resource supply b_k, as dict[k] or list
    M_prec  : big-M for precedence constraints
    M_flow  : big-M for capacity constraints 0 <= f_ijk <= M_flow * x_ij

    Returns:
        m    : Gurobi model
        x    : x[i,j] binary policy arcs
        S    : dict of S[(h,i)] variables
        rho  : robust makespan variable
        f    : f[i,j,k] flow variables
    """
    m = gp.Model("master_rcpsp")
    if not gurobi_output:
        m.Params.OutputFlag = 0

    source = V[0]
    sink = V[-1]
    H = len(P_list)        # number of scenarios
    nV = len(V)

    # --- Decision variables ---

    # x_ij ∈ {0,1}
    x = m.addVars(V, V, vtype=GRB.BINARY, name="x")

    # S^h_i ≥ 0  (we store them in a dict S[(h,i)])
    S = {}
    for h, p in enumerate(P_list):
        for i in V:
            S[(h, i)] = m.addVar(lb=0.0, name=f"S[{h},{i}]")

    # ρ ≥ 0
    rho = m.addVar(lb=0.0, name="rho")

    # f_ijk ≥ 0
    f = m.addVars(V, V, R, lb=0.0, name="f")

    # --- Objective ---
    # min ρ
    m.setObjective(rho, GRB.MINIMIZE)

    # --- Constraints ---

    # (eq. rho_ge_S)  ρ ≥ S^h_{sink}  for all scenarios h
    for h in range(H):
        m.addConstr(rho >= S[(h, sink)], name=f"rho_lb[{h}]")

    # (eq. prec_scen)  S^h_j ≥ S^h_i + p^h_i - M(1 - x_ij)
    for h, p in enumerate(P_list):
        for i in V:
            p_hi = float(p[i])
            for j in V:
                if i == j:
                    continue
                m.addConstr(
                    S[(h, j)] >= S[(h, i)] + p_hi - M_prec * (1 - x[i, j]),
                    name=f"prec[{i},{j},{h}]"
                )

    # (eq. nonneg_S) is enforced by lb=0.0 on S^h_i

    # (eq. f_nonneg) is enforced by lb=0.0 on f_ijk

    # (eq. flow_conserv)  internal nodes: sum_j f_{j i k} = sum_j f_{i j k} = b_{ik}
    for k in R:
        for i in V:
            if i in (source, sink):
                continue
            inflow = gp.quicksum(f[j, i, k] for j in V if j != i)
            outflow = gp.quicksum(f[i, j, k] for j in V if j != i)
            b_ik = _get_b_node(b_node, i, k)
            m.addConstr(inflow == b_ik, name=f"flow_in[{i},{k}]")
            m.addConstr(outflow == b_ik, name=f"flow_out[{i},{k}]")

    # (eq. flow_supply)  source/sink: sum_j f_{0 j k} = sum_j f_{j,n+1,k} = b_k
    for k in R:
        b_k = _get_b_supply(b_supply, k)
        m.addConstr(
            gp.quicksum(f[source, j, k] for j in V if j != source) == b_k,
            name=f"supply_source[{k}]"
        )
        m.addConstr(
            gp.quicksum(f[j, sink, k] for j in V if j != sink) == b_k,
            name=f"supply_sink[{k}]"
        )

    # (eq. cap_flow)  0 ≤ f_{ijk} ≤ M * x_ij
    for k in R:
        for i in V:
            for j in V:
                if i == j:
                    # if you want, you can force no self-flow:
                    m.addConstr(f[i, j, k] == 0.0, name=f"no_self_flow[{i},{k}]")
                else:
                    m.addConstr(
                        f[i, j, k] <= M_flow * x[i, j],
                        name=f"cap[{i},{j},{k}]"
                    )

    # (eq. x_fixed)  x_{ij} = 1 for (i,j) in E
    for (i, j) in E:
        m.addConstr(x[i, j] == 1, name=f"x_fixed[{i},{j}]")

    # (eq. x_bin) is enforced by vtype=GRB.BINARY

    m.update()
    return m, x, S, rho, f


In [12]:
def add_scenario_to_master(
    m,
    x,
    S,
    rho,
    p_vec,
    h_new,
    V,
    M_prec=1e6,
):
    """
    Add a new scenario h_new with durations p_vec to an existing master model.

    m     : existing Gurobi model
    x     : x[i,j] binary vars (shared across scenarios)
    S     : dict of S[(h,i)] vars; this function will extend it
    rho   : robust makespan var
    p_vec : list/array of durations p^{h_new}_i (len == len(V))
    h_new : integer index of the new scenario (e.g. current number of scenarios)
    V     : list of nodes
    M_prec: big-M for precedence constraints
    """
    sink = V[-1]

    # Create S^{h_new}_i ≥ 0 for all i
    for i in V:
        S[(h_new, i)] = m.addVar(lb=0.0, name=f"S[{h_new},{i}]")

    # Add constraint ρ ≥ S^{h_new}_{sink}
    m.addConstr(rho >= S[(h_new, sink)], name=f"rho_lb[{h_new}]")

    # Add precedence constraints for scenario h_new:
    # S^{h_new}_j ≥ S^{h_new}_i + p^{h_new}_i - M*(1 - x_ij)
    for i in V:
        p_hi = float(p_vec[i])
        for j in V:
            if i == j:
                continue
            m.addConstr(
                S[(h_new, j)] >= S[(h_new, i)] + p_hi - M_prec * (1 - x[i, j]),
                name=f"prec[{i},{j},{h_new}]"
            )

    m.update()


In [13]:
import gurobipy as gp
from gurobipy import GRB

# --------- Build a tiny example instance ---------
V = [0, 1, 2, 3]      # 0 = source, 3 = sink
E = [(0, 1), (1, 2), (2, 3)]  # fixed precedence path

# Initial scenario set P_list = {p^1}
p1 = [2.0, 3.0, 1.5, 0.0]   # durations for nodes 0..3
P_list = [p1]

# Single resource k = 0
R = [0]

# b_node[i][k]: for i=1,2 we want 1 unit of flow in and out; source/sink: 0
b_node = [
    [0.0],   # i=0 (source)
    [1.0],   # i=1 (internal)
    [1.0],   # i=2 (internal)
    [0.0],   # i=3 (sink)
]

# b_supply[k]: 1 unit from source to sink
b_supply = [1.0]

# --------- Create and solve the master problem ---------
m, x, S, rho, f = create_master_problem(
    V=V,
    E=E,
    P_list=P_list,
    R=R,
    b_node=b_node,
    b_supply=b_supply,
    M_prec=1e4,
    M_flow=1e4,
    gurobi_output=True,  # or False if you want silence
)

m.optimize()

if m.Status != GRB.OPTIMAL:
    raise RuntimeError(f"Master not optimal, status={m.Status}")

print("\n=== MASTER (only p^1) ===")
print("rho* =", rho.X)

# Policy edges (where x_ij ≈ 1)
def extract_policy_edges_from_x(x, V, tol=0.5):
    edges = []
    for i in V:
        for j in V:
            if i != j and x[i, j].X is not None and x[i, j].X > tol:
                edges.append((i, j))
    return edges

policy_edges = extract_policy_edges_from_x(x, V)
print("Policy edges X:", policy_edges)

print("\nS values for h=0:")
for i in V:
    print(f"S[0,{i}] =", S[(0, i)].X)

print("\nFlow values f[i,j,0] > 1e-6:")
for i in V:
    for j in V:
        val = f[i, j, 0].X
        if val > 1e-6:
            print(f"f[{i},{j},0] =", val)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 38 rows, 37 columns and 87 nonzeros
Model fingerprint: 0x18f6cb42
Variable types: 21 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 6.5000000
Presolve removed 26 rows and 33 columns
Presolve time: 0.02s
Presolved: 12 rows, 4 columns, 24 nonzeros
Variable types: 4 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 6.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.500000000000e+00, best bound 6.500000000000e

In [14]:
# New scenario p^2
p2 = [2.5, 4.0, 2.0, 0.0]
h_new = 1  # because existing h=0, so this is the second scenario

add_scenario_to_master(
    m=m,
    x=x,
    S=S,
    rho=rho,
    p_vec=p2,
    h_new=h_new,
    V=V,
    M_prec=1e4,
)

m.optimize()

if m.Status != GRB.OPTIMAL:
    raise RuntimeError(f"Master not optimal after adding p^2, status={m.Status}")

print("\n=== MASTER (after adding p^2) ===")
print("rho* =", rho.X)

policy_edges = extract_policy_edges_from_x(x, V)
print("Policy edges X:", policy_edges)

print("\nS values for h=0 and h=1:")
for h in [0, 1]:
    for i in V:
        print(f"S[{h},{i}] =", S[(h, i)].X)
    print("---")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 51 rows, 41 columns and 125 nonzeros
Model fingerprint: 0xd13efd9b
Variable types: 25 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective 8.5 (0.01s)
Loaded MIP start from previous solve with objective 8.5

Presolve removed 26 rows and 32 columns
Presolve time: 0.00s
Presolved: 25 rows, 9 columns, 51 nonzeros
Variable types: 9 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 12 